# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [39]:
import re
import time
import typing
import requests

from random import randint
import functools
from typing import TypeVar
from collections.abc import Callable
from typing import TypeVar
from typing_extensions import  ParamSpec

In [40]:
P = ParamSpec("P")
T = TypeVar("T")

In [41]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [55]:
def benchmark(func : Callable[P, T]) -> Callable[P, T]:
    @functools.wraps(func)
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> T:
        start_time = time.time()
        r = func(*args, **kwargs)
        end_time = time.time()
        print(f"Время выполнения функции {func.__name__}: {end_time - start_time}", end="\n\n")
        return r
    return wrapper

## Задание 2

In [56]:
def logging(func: Callable[P, T]) -> Callable[P, T]:
    @functools.wraps(func)
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> T:
        r = func(*args, **kwargs)
        print("Функция вызвана с параметрами:\n"
              f"{args}, {kwargs}", end="\n\n")
        return r
    return wrapper

## Задание 3

In [57]:
def counter(func: Callable[P, T]) -> Callable[P, T]:
    calls: int = 0
    @functools.wraps(func)
    def wrapper(*args: P.args, **kwargs: P.kwargs):
        r = func(*args, **kwargs)
        nonlocal calls
        calls += 1
        print(f"Функция была вызвана: {calls} раз\n")
        return r
    return wrapper

## Задание 4

In [130]:
def memo(func: Callable[P, T]) -> Callable[P, T]:
  cache: dict = dict()
  @functools.wraps(func)
  def fmemo(*args):
    for arg in args:
        if not isinstance(arg, typing.Hashable):
            raise Exception("Передан нехэшируемый аргумент") 
          
    if not args in cache:
        cache[args] = func(*args)
    return cache[args]

  fmemo.cache = cache
  return fmemo

## Тестирование

In [59]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Время выполнения функции word_count: 3.848121166229248

Функция вызвана с параметрами:
('whole',), {}

Функция была вызвана: 1 раз

Cлово whole встречается 176 раз


In [123]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [124]:
%%timeit
# измеряем время выполнения
fib(25)

26.8 ms ± 7.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [132]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [128]:
%%timeit
# измеряем время выполнения
fib(25)

946 ns ± 25.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Вывод: с декоратором метод работает в ~260 000 раз быстрее